In [ ]:
import pandas as pd
from prophet import Prophet
from pyspark.sql.functions import month, date_format
from pyspark.sql.types import IntegerType
import warnings
warnings.filterwarnings('ignore')

spark = SparkSession.builder.appName(
    'Read from Big Query into DataFrame').getOrCreate()

In [ ]:
df = spark.read \
  .format("bigquery") \
  .load("<your-project-name-here>.biglake_dataset.IceCreamSales")

In [ ]:
df.show(10)

In [ ]:
cleaneddDf = df.select(date_format('Month', 'MM/yyyy')
                       .alias('ds'), "Gross_Revenue").withColumn("Revenue", df["Gross_Revenue"].cast(IntegerType()))
groupedDf = cleaneddDf.groupBy('ds').sum("Revenue").withColumnRenamed("sum(Revenue)", "y")
history = groupedDf.toPandas()

In [ ]:
model = Prophet(
    interval_width=0.90
)

In [ ]:
model.fit(history)

In [ ]:
future_date = model.make_future_dataframe(
    periods=6,
    freq='m',
    include_history=True
)

In [ ]:
forecast = model.predict(future_date)

In [ ]:
predict_chart = model.plot(forecast, xlabel='date', ylabel='sales')

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()